# Setup

In [1]:
import os
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

In [ ]:
# dataset root directory:
DATA_DIR = "/Users/eugenekim/Emo-CLIM/dataset/DeepEmotion"
# label subdirectories:
SUBDIR_NAMES = ["excitement", "anger", "fear", "amusement", "awe", "contentment", "disgust", "sadness"]

In [ ]:
# script options:
val_fract = 0.1
test_fract = 0.1
metadata_files_split = {
    "train": "/Users/eugenekim/Emo-CLIM/dataset/DeepEmotion/metadata_train.csv",
    "val": "/Users/eugenekim/Emo-CLIM/dataset/DeepEmotion/metadata_val.csv",
    "test": "/Users/eugenekim/Emo-CLIM/dataset/DeepEmotion/metadata_test.csv"
}

# Get Image Files

In [ ]:
# get dataset label directories:
subdir_names = [name for name in os.listdir(DATA_DIR) if os.path.isdir(os.path.join(DATA_DIR, name))]
subdir_names = [name for name in subdir_names if name in SUBDIR_NAMES]
assert set(subdir_names) == set(SUBDIR_NAMES), f"Error: Found {subdir_names}, expected {SUBDIR_NAMES}"
# Use SUBDIR_NAMES for consistent ordering
subdir_names = SUBDIR_NAMES

In [5]:
# get all image file paths:
subdirs = []
image_file_names = []
emotion_labels = []
for label in subdir_names:
    # get file_names:
    subdir_path = os.path.join(DATA_DIR, label)
    file_names = [name for name in os.listdir(subdir_path) if os.path.isfile(os.path.join(subdir_path, name))]
    n_images = len(file_names)
    # save metadata:
    subdirs += n_images * [label]
    image_file_names += file_names
    emotion_labels += n_images * [label]

# create dataframe:
all_metadata = pd.DataFrame(
    data={
        "subdir_name": subdirs,
        "file_name": image_file_names,
        "label": emotion_labels
    }
)

print("Total size of dataset: {}".format(all_metadata.shape[0]))
print()
print(all_metadata.info())

Total size of dataset: 21829

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21829 entries, 0 to 21828
Data columns (total 3 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   subdir_name  21829 non-null  object
 1   file_name    21829 non-null  object
 2   label        21829 non-null  object
dtypes: object(3)
memory usage: 511.7+ KB
None


In [6]:
# print subset sizes:
train_set_size = int(np.around((1 - test_fract - val_fract) * all_metadata.shape[0]))
print("Training set size: {:.2f}% of dataset = {} samples.".format(100 * (1 - test_fract - val_fract), train_set_size))
val_set_size = int(np.around(val_fract * all_metadata.shape[0]))
print("Validation set size: {:.2f}% of dataset = {} samples.".format(100 * val_fract, val_set_size))
test_set_size = int(np.around(test_fract * all_metadata.shape[0]))
print("Test set size: {:.2f}% of dataset = {} samples.".format(100 * test_fract, test_set_size))

Training set size: 80.00% of dataset = 17463 samples.
Validation set size: 10.00% of dataset = 2183 samples.
Test set size: 10.00% of dataset = 2183 samples.


In [7]:
# print class distribution:
label_counts = all_metadata["label"].value_counts()
for label in all_metadata["label"].value_counts().index:
    print("Number of {} images: {}".format(label, label_counts[label]))

Number of contentment images: 5130
Number of amusement images: 4724
Number of awe images: 2881
Number of excitement images: 2725
Number of sadness images: 2633
Number of disgust images: 1591
Number of anger images: 1176
Number of fear images: 969


# Split Dataset

In [8]:
# split into stratified training/val and test sets:
metadata_train_val, metadata_test = train_test_split(all_metadata, test_size=test_set_size, stratify=all_metadata["label"], random_state=42)
assert metadata_test.shape[0] == test_set_size, "Test set metadata has incorrect size."
print(metadata_test.info())

# print test set class distribution:
print()
label_counts = metadata_test["label"].value_counts()
for label in metadata_test["label"].value_counts().index:
    print("Number of {} images: {}".format(label, label_counts[label]))

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2183 entries, 19379 to 2890
Data columns (total 3 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   subdir_name  2183 non-null   object
 1   file_name    2183 non-null   object
 2   label        2183 non-null   object
dtypes: object(3)
memory usage: 68.2+ KB
None

Number of contentment images: 513
Number of amusement images: 472
Number of awe images: 288
Number of excitement images: 273
Number of sadness images: 263
Number of disgust images: 159
Number of anger images: 118
Number of fear images: 97


In [9]:
# split into stratified training and validation sets:
metadata_train, metadata_val = train_test_split(metadata_train_val, test_size=val_set_size, stratify=metadata_train_val["label"], random_state=42)
assert metadata_val.shape[0] == val_set_size, "Validation set metadata has incorrect size."
print(metadata_val.info())

# print validation set class distribution:
print()
label_counts = metadata_val["label"].value_counts()
for label in metadata_val["label"].value_counts().index:
    print("Number of {} images: {}".format(label, label_counts[label]))

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2183 entries, 19808 to 7695
Data columns (total 3 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   subdir_name  2183 non-null   object
 1   file_name    2183 non-null   object
 2   label        2183 non-null   object
dtypes: object(3)
memory usage: 68.2+ KB
None

Number of contentment images: 513
Number of amusement images: 473
Number of awe images: 288
Number of excitement images: 272
Number of sadness images: 263
Number of disgust images: 159
Number of anger images: 118
Number of fear images: 97


In [10]:
# check that all subsets are disjoint:
metadata_subsets = [metadata_train, metadata_val, metadata_test]
subset_names = list(metadata_files_split.keys())
for subset_1, name_1 in zip(metadata_subsets, subset_names):
    for subset_2, name_2 in zip(metadata_subsets, subset_names):
        if name_1 != name_2:
            assert set(subset_1.index).isdisjoint(set(subset_2.index)), "{} and {} are not disjoint".format(name_1, name_2)

In [11]:
# reset indices:
metadata_train = metadata_train.reset_index(drop=True)
metadata_val = metadata_val.reset_index(drop=True)
metadata_test = metadata_test.reset_index(drop=True)

# sanity checks:
assert all_metadata.shape[0] == metadata_train.shape[0] + metadata_val.shape[0] + metadata_test.shape[0], "Subset set sizes don't add up."
# check that all subsets are disjoint:
metadata_subsets = [metadata_train, metadata_val, metadata_test]
subset_names = list(metadata_files_split.keys())
for subset_1, name_1 in zip(metadata_subsets, subset_names):
    for subset_2, name_2 in zip(metadata_subsets, subset_names):
        if name_1 != name_2:
            assert set(subset_1["file_name"].tolist()).isdisjoint(set(subset_2["file_name"].tolist())), "{} and {} are not disjoint".format(name_1, name_2)
# more sanity checks:
class_counts_all = all_metadata["label"].value_counts()
class_counts_train = metadata_train["label"].value_counts()
class_counts_val = metadata_val["label"].value_counts()
class_counts_test = metadata_test["label"].value_counts()
for class_label in all_metadata["label"].unique().tolist():
    assert class_counts_all[class_label] == class_counts_train[class_label] + class_counts_val[class_label] + class_counts_test[class_label], "Error with splitting dataset."

# save to file:
metadata_train.to_csv(metadata_files_split["train"], index=False)
metadata_val.to_csv(metadata_files_split["val"], index=False)
metadata_test.to_csv(metadata_files_split["test"], index=False)